In [12]:
import time
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Парсинг рейтингов университетов

Одна из самых популярных пузомерок — рейтинги университетов. Есть российские, есть зарубежные. В данном материале собраны «рецепты» парсинга следующих рейтингов:

1. Московский рейтинг университетов — три миссии университета
2. Рейтинг университетов Интерфакса
3. RAEX 100, в т.ч. по предметам
4. Рейтинг Best Global Universities, в т.ч. по предметам
5. QS-рейтинг университетов, в т.ч. по предметам
6. Шанхайский рейтинг университетов
7. Рейтинг университетов от Times Higher Education

Рейтинги выходят в разное время года. Какие-то уже обновились и есть данные за 2023 год, а какие-то заканчиваются 2022 годом. Чтобы исключить лишнюю вариативность в данном обучающем материале ограничим диапазон тремя календарными годами.

In [5]:
years = list(range(2020, 2022 + 1))
years

[2020, 2021, 2022]

## Московский рейтинг университетов

In [16]:
msk_home_url = 'https://mosiur.org'
msk_last_year = 2022
msk_data = []

for y in years:
    msk_url = f"{msk_home_url}/{'ranking' if y == msk_last_year else 'ranking' + str(y)}/"
    msk_resp = requests.get(msk_url)

    if msk_resp.status_code == 200:
        msk_soup = BeautifulSoup(msk_resp.text)

        for tr in msk_soup.select('#top_table tbody tr'):
            tds = tr.find_all('td')
            msk_data.append({
                'year': y,
                'rank': tds[0].text,
                'university': tds[1].text,
                'country': tds[2].text
            })
    else:
        print(f"{msk_url} return status code {msk_resp.status_code}")

    # Бережём источник данных
    time.sleep(1)

msk_df = pd.DataFrame(msk_data)
msk_df.to_csv('data/msk.csv', index=False)
msk_df.to_excel('data/msk.xlsx', index=False)
msk_df.head(5)

,year,rank,university,country
0,2020,1,Harvard University,США
1,2020,2,Massachusetts Institute of Technology,США
2,2020,3,University of Cambridge,Великобритания
3,2020,4,University of Oxford,Великобритания
4,2020,5,University of Pennsylvania,США


Данные Московского рейтинга получены. Давайте посмотрим представительство стран по годам (ТОП 10):

In [21]:
msk_pivot = pd.pivot_table(
    msk_df,
    index='country',
    columns='year',
    values='university',
    aggfunc='count'
)
msk_pivot['sum'] = msk_pivot.sum(axis=1)
msk_pivot.sort_values(by='sum', ascending=False).head(10)

year,2020,2021,2022,sum
country,,,,
США,220.0,239.0,253.0,712.0
Китай,122.0,144.0,173.0,439.0
Россия,101.0,112.0,146.0,359.0
Великобритания,98.0,106.0,108.0,312.0
Япония,93.0,102.0,101.0,296.0
Германия,69.0,73.0,74.0,216.0
Италия,50.0,54.0,54.0,158.0
Индия,45.0,51.0,58.0,154.0
Испания,42.0,55.0,56.0,153.0


## Рейтинг Интерфакса

In [24]:
def get_interfax_list_by_year(year, page=1):
    url = f"https://academia.interfax.ru/data/rating/?rating=1&year={year}&page={page}"
    result = []
    resp = requests.get(url)

    # Бережём источник данных
    time.sleep(1)

    if resp.status_code == 200:
        json_data = resp.json()
        result += json_data['universities']
        page_count = json_data['page_count']

        if page != page_count:
            result += get_interfax_list_by_year(year, page + 1)
    else:
        print(f"{url} return status code {resp.status_code}")

    return result


interfax_df = pd.DataFrame()

for y in years:
    interfax_df_by_year = pd.DataFrame(get_interfax_list_by_year(y))
    interfax_df_by_year['year'] = y
    interfax_df = pd.concat([interfax_df, interfax_df_by_year], ignore_index=True)

interfax_df.to_csv('data/interfax.csv', index=False)
interfax_df.to_excel('data/interfax.xlsx', index=False)
interfax_df.head()

,change,description,is_close,point,rank,name,id,url,year
0,0,None,False,1000,1,Московский государственный университет имени М...,1,https://www.msu.ru,2020
1,0,None,False,963,2,Национальный исследовательский ядерный универс...,2,https://mephi.ru,2020
2,0,None,False,961,3,Московский физико-технический институт (госуда...,4,https://mipt.ru,2020
3,0,None,False,857,4,Национальный исследовательский университет «Вы...,6,https://www.hse.ru,2020
4,1,None,False,848,5,Новосибирский национальный исследовательский г...,3,http://www.nsu.ru/?lang=ru,2020


В данных рейтинга Интерфакса уже есть данные про движения в рамках рейтинга. Давайте посмотрим на ТОП 10 университетов, которые поднялись в рейтинге за 3 последних года больше всех.

In [27]:
interfax_pivot = pd.pivot_table(
    interfax_df,
    index='name',
    columns='year',
    values='change',
    aggfunc='sum'
)
interfax_pivot['sum'] = interfax_pivot.sum(axis=1)
interfax_pivot.sort_values(by='sum', ascending=False).head(10)

year,2020,2021,2022,sum
name,,,,
Уральский государственный аграрный университет,-21.0,40.0,104.0,123.0
Московский институт психоанализа,-8.0,3.0,128.0,123.0
Университет Синергия,-1.0,78.0,43.0,120.0
Чеченский государственный университет имени А.А.Кадырова,-22.0,83.0,58.0,119.0
Московский государственный психолого-педагогический университет,15.0,78.0,18.0,111.0
Красноярский государственный аграрный университет,-1.0,-12.0,122.0,109.0
Башкирский государственный педагогический университет имени М.Акмуллы,-34.0,45.0,93.0,104.0
Кубанский государственный технологический университет,24.0,18.0,61.0,103.0
Московский государственный областной университет,5.0,28.0,51.0,84.0
